# Timelags: Bundle Heating Model
Compute timelags from intensities computed using the bundle heating model. This is all still a PoC and is only for the future work section of the conclusions.

In [1]:
import os
import sys

import numpy as np
import distributed
import matplotlib.pyplot as plt
import matplotlib.colors
from sunpy.map import Map,GenericMap
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.utils.console import ProgressBar

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.analysis import DistributedAIACube,AIATimelags
from synthesizAR.visualize import bgry_004_idl_cmap

%matplotlib inline

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

In [2]:
cluster = distributed.LocalCluster(n_workers=16,threads_per_worker=4,)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39925 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 64 Memory: 270.38 GB


In [3]:
channels = [94,131,171,193,211,335]
channel_pairs = [(94,335),
                 (94,171),
                 (94,193),
                 (94,131),
                 (94,211),
                 (335,131),
                 (335,193),
                 (335,211), 
                 (335,171),
                 (211,131),
                 (211,171),
                 (211,193),
                 (193,171),
                 (193,131),
                 (171,131),]

In [4]:
intensity_file_format = '/storage-home/w/wtb2/data/bundle_heating_model/SDO_AIA/{}/map_t{:06d}.fits'

In [5]:
result_file_format = '/storage-home/w/wtb2/projects/dissertation/thesis/chapter8/data/bundle-model/{}_{}_{}.fits'

In [6]:
timelag_bounds = (-6*u.hour, 6*u.hour)

In [7]:
tl = AIATimelags(
    *[DistributedAIACube.from_files([intensity_file_format.format(c, i) for i in range(2000)])
      for c in channels])

In [8]:
chunks = (tl[0].shape[1]//5, tl[1].shape[2]//5)

In [9]:
for ca,cb in channel_pairs:
    timelag_map = tl.timelag_map(f'{ca}',f'{cb}', timelag_bounds=timelag_bounds,chunks=chunks)
    timelag_map.save(result_file_format.format('timelag', ca, cb),overwrite=True)
    correlation_map = tl.peak_cross_correlation_map(f'{ca}',f'{cb}', timelag_bounds=timelag_bounds,chunks=chunks)
    correlation_map.save(result_file_format.format('correlation', ca, cb),overwrite=True)